In [25]:
#%pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display
from queue import Queue
from threading import Thread

# Definição de filas para mensagens e gravações
messages = Queue()
recordings = Queue()

# Botão para iniciar a gravação
record_button = widgets.Button(
    description='Gravar',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

# Botão para parar a gravação
stop_button = widgets.Button(
    description='Parar',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

output = widgets.Output()

# Função para iniciar a gravação
def start_recording(data):
    messages.put(True)
    
    with output:
        display("Gravando...")
        record = Thread(target=record_microphone)
        record.start()
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()

# Função para parar a gravação
def stop_recording(data):
    with output:
        messages.get()
        display("Finalizado.")
    
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

Button(button_style='success', description='Gravar', icon='microphone', style=ButtonStyle(), tooltip='Record')

Button(button_style='warning', description='Parar', icon='stop', style=ButtonStyle(), tooltip='Stop')

Output()

In [26]:
import os

# Obtém e imprime o diretório de trabalho atual
os.getcwd()


'c:\\Users\\Pichau\\Desktop\\Analise_Esportiva'

In [27]:
#pip install pyaudio

import pyaudio 

p = pyaudio.PyAudio()
 
 # Imprimir os dispositivos de áudio da sua máquina
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))
    
    
p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microfone (2- USB Microphone)', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microfone (2- Realtek High Defi', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannel

In [28]:
# Definição das configurações de áudio
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

# Função para gravar áudio do microfone
def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate=FRAME_RATE,
                    input=True,
                    input_device_index=2, # Defina qual o número do mic através do print anterior
                    frames_per_buffer=chunk)

    frames = []

    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()

In [29]:
# %pip install vosk
# %pip install transformers
# %pip install torch

In [30]:
import subprocess
import json
from vosk import Model, KaldiRecognizer
import time

# Carregamento do modelo Vosk para PT-BR
model = Model(model_name="vosk-model-small-pt-0.3") #vosk-model-en-us-0.22
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)
    
# Função para reconhecimento de fala
def speech_recognition(output):
    
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        output.append_stdout(text)
        time.sleep(1)
        
        # Pontuações
        # cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)
        # cased = subprocess.check_output('python c:\\Users\\Pichau\\Desktop\\Analise_Esportiva\\recasepunc\\recasepunc.py predict c:\\Users\\Pichau\\Desktop\\Analise_Esportiva\\recasepunc\\checkpoint', shell=True, text=True, input=text)
        # output.append_stdout(cased)
        # time.sleep(1)
                      
#             #https://github.com/falabrasil/kaldi-br
            
          #   cased = subprocess.check_output('python c:\\Users\\Pichau\\Desktop\\Analise_Esportiva\\recasepunc\\recasepunc.py predict c:\\Users\\Pichau\\Desktop\\Analise_Esportiva\\recasepunc\\checkpoint', shell=True, text=True, input=text)
          #   output.append_stdout(cased)
          